In [1]:
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/MO_pipeline/')
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS/')
import scendict as sc
import utils as ut
import netCDF4 as nc
import xarray as xr
import numpy as np
from importlib import reload
reload(sc)

import cartopy as cp
import cartopy.crs as ccrs
import cmocean as cm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import glob
plt.rcParams.update({'font.size': 14})
font = {'family' : 'normal',
'weight' : 'bold',
'size'   : 14}

### Load data for Cflx and windspeed 

In [2]:
def extract_era_from_daily_2D(yrstart,yrend):

    yrs = np.arange(yrstart,yrend+1,1)
    storar = np.zeros([len(yrs),12,37,182])
    for i in range(0,len(yrs)):
        yr = yrs[i]
        eradir = '/gpfs/home/mep22dku/scratch/SOZONE/prelim_EVAL_forcing/averaged_forcing/ERA5/'
        wy = glob.glob(f'{eradir}/ERA5_MONTHLY_SO_wind{yr}.nc')
        try:
            wy2 = nc.Dataset(wy[0])
            storar[i,:,:,:]=  (wy2['wspd10m'][:,:,:])
        except:
            storar[i,:,:,:] = np.nan
            
    return storar

def extract_ncep_from_daily_2D(yrstart,yrend):

    yrs = np.arange(yrstart,yrend+1,1)
    storar = np.zeros([len(yrs),12,37,182])
    for i in range(0,len(yrs)):
        yr = yrs[i]
        eradir = '/gpfs/home/mep22dku/scratch/SOZONE/prelim_EVAL_forcing/averaged_forcing/NCEP/'
        wy = glob.glob(f'{eradir}/NCEP_MONTHLY_SO_wind{yr}.nc')
        try:
            
            wy2 = nc.Dataset(wy[0])
            storar[i,:,:,:]=  (wy2['wspd10m'][:,:,:])
        except:
            storar[i,:,:,:] = np.nan
            
    return storar

In [3]:
# help(ut.extract_model_from_daily_2D)
### CFLX for MET OFFICE
Cflx_1AS1 = ut.extract_model_from_daily_2D(1940,2100,\
            '/gpfs/afm/greenocean/software/runs/TOM12_TJ_1AS1/',\
             'diad_T', 'Cflx')
Cflx_1AW1 = ut.extract_model_from_daily_2D(1940,2100,\
            '/gpfs/afm/greenocean/software/runs/TOM12_TJ_1AW1/',\
             'diad_T', 'Cflx')
w = nc.Dataset('../MO_pipeline/EXTRACT/ncs/scen_1A_monthly_wspd.nc')
wspd_1AS1 = w['mean_wspd'][:,0:12,0:37,:]

### for ERA5
Cflx_WE43 = ut.extract_model_from_daily_2D(1940,2100,\
            '/gpfs/data/greenocean/software/runs/TOM12_DW_WE43/',\
             'diad_T', 'Cflx')
Cflx_WE45 = ut.extract_model_from_daily_2D(1940,2100,\
            '/gpfs/data/greenocean/software/runs/TOM12_DW_WE45/',\
             'diad_T', 'Cflx')
wspd_WE43 = extract_era_from_daily_2D(1940,2101)

### for NCEP
Cflx_WD30 = ut.extract_model_from_daily_2D(1940,2100,\
            '/gpfs/data/greenocean/software/runs/TOM12_DW_WD30/',\
             'diad_T', 'Cflx')
Cflx_WD32 = ut.extract_model_from_daily_2D(1940,2100,\
            '/gpfs/data/greenocean/software/runs/TOM12_DW_WD32/',\
             'diad_T', 'Cflx')
wspd_WD30 = extract_ncep_from_daily_2D(1940,2101)

#### get it all the same shape and extract timeseries (by-lat hovmollers)

In [4]:
def lat_hovmoller(ar):
    ar_bylat = np.nanmean(ar,axis = 3)
    ar_hov = np.zeros([np.shape(ar_bylat)[0]*np.shape(ar_bylat)[1],np.shape(ar_bylat)[2]])
    ind = 0
    #no of years
    for i in range(0,np.shape(ar_bylat)[0]):
        ar_hov[ind:ind+12,:] = ar_bylat[i,:,:]
        ind = ind+12
        
    return ar_hov

In [5]:
# get it all the same shape
Cflx_1AS1 = Cflx_1AS1[:,:,0:37,:]
Cflx_1AW1 = Cflx_1AW1[:,:,0:37,:]
Cflx_1ASW = Cflx_1AS1-Cflx_1AW1

Cflx_WE43 = Cflx_WE43[:,:,0:37,:]
Cflx_WE45 = Cflx_WE45[:,:,0:37,:]
Cflx_43SW = Cflx_WE43-Cflx_WE45

Cflx_WD30 = Cflx_WD30[:,:,0:37,:]
Cflx_WD32 = Cflx_WD32[:,:,0:37,:]
Cflx_30SW = Cflx_WD30-Cflx_WD32


Cflx_WD30_hov = lat_hovmoller(Cflx_WD30)
Cflx_WD32_hov = lat_hovmoller(Cflx_WD32)
Cflx_30SW_hov = lat_hovmoller(Cflx_30SW)

Cflx_1AS1_hov = lat_hovmoller(Cflx_1AS1)
Cflx_1AW1_hov = lat_hovmoller(Cflx_1AW1)
Cflx_1ASW_hov = lat_hovmoller(Cflx_1ASW)

Cflx_WE43_hov = lat_hovmoller(Cflx_WE43)
Cflx_WE45_hov = lat_hovmoller(Cflx_WE45)
Cflx_43SW_hov = lat_hovmoller(Cflx_43SW)

wspd_1AS1_hov = lat_hovmoller(wspd_1AS1)
wspd_WE43_hov = lat_hovmoller(wspd_WE43)
wspd_WD30_hov = lat_hovmoller(wspd_WD30)

yrs = np.arange(1940,2101,(1/12))
yrs = np.round(yrs, 3)
# horrible indexing
def start_end(si,ei):
    s = (np.where(np.abs(yrs-si) < 0.00001)); s = s[0][0]
    e = (np.where(np.abs(yrs-ei) < 0.00001)); e = e[0][0]
    return s,e


In [6]:
s,e = start_end(1970,2020)
fact = 0.6
 
tvmin = -1.5e-7; tvmax = 1.5e-7; tcmap = cm.cm.balance_r
fig, axs = plt.subplots(3,3, figsize=(24*fact, 18*fact), \
                        facecolor='w', edgecolor='k')
winds = 
axs = axs.ravel()
axs
#wind anomaly hovmoller, #air-sea flux hovmoller, #co2 anomaly hovmoller, 
axs

SyntaxError: invalid syntax (3862494454.py, line 7)

In [ ]:
import numpy as np
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'same') / w

data = np.array([2,2,2,2,2,2,4,4,4,4,4,4])

print(moving_average(data,5))

plt.plot(data, label = 'dat')
plt.plot(moving_average(data,5))
print(np.shape(data))
print(np.shape(moving_average(data,5)))
plt.legend()

In [ ]:
#get out one location
test_cflx = Cflx_WE43[:,:,25,40]
test_wspd = wspd_WE43[:,:,25,40]**2 #windspeed squared

#monthly cflx
test_cflx_m = np.nanmean(test_cflx,axis = 0)
#broadcast to same-shape array and ravel
test_cflx_m_b = np.zeros_like(test_cflx)
for i in range(0,12):
    test_cflx_m_b[:,i] = test_cflx_m[i]
test_cflx_r = np.ravel(test_cflx, order = 'C')
test_cflx_m_r = np.ravel(test_cflx_m_b)
#calculate the moving average - careful, the edges will be wonky, but you don't care
test_cflx_rm = moving_average(test_cflx_r,12)

#
#monthly wspd
test_wspd_m = np.nanmean(test_wspd,axis = 0)
#broadcast to same-shape array and ravel
test_wspd_m_b = np.zeros_like(test_wspd)
for i in range(0,12):
    test_wspd_m_b[:,i] = test_wspd_m[i]
test_wspd_r = np.ravel(test_wspd, order = 'C')
test_wspd_m_r = np.ravel(test_wspd_m_b)
#calculate the moving average - careful, the edges will be wonky, but you don't care
test_wspd_rm = moving_average(test_wspd_r,12)


s,e=start_end(1970,2020)

fact = 0.8
fig, axs = plt.subplots(3,1, figsize=(24*fact, 20*fact), \
                        facecolor='w', edgecolor='k')
axs = axs.ravel()
axs[0].plot(yrs[s:e],test_cflx_r[s:e], label = 'cflx')
axs[0].plot(yrs[s:e],test_cflx_m_r[s:e], label = 'cflx monthly mean')
axs[0].plot(yrs[s:e],test_cflx_rm[s:e], label = 'cflx running mean')
cflx_anom = test_cflx_r[s:e]-(test_cflx_m_r[s:e]+test_cflx_rm[s:e])
cflx_anom2 = test_cflx_r[s:e]-(test_cflx_m_r[s:e])
axs[0].plot(yrs[s:e],cflx_anom, linewidth = 2, color = 'r',\
            label = 'CFLX ANOMALY \n cflx - (monthly mean + running mean)')
# axs[0].plot(yrs[s:e],cflx_anom2, linewidth = 2, color = 'darkred',\
#             label = 'CFLX ANOMALY \n cflx - (monthly mean)')

axs[1].plot(yrs[s:e],test_wspd_r[s:e], label = 'wspd')
axs[1].plot(yrs[s:e],test_wspd_m_r[s:e], label = 'wspd monthly mean')
axs[1].plot(yrs[s:e],test_wspd_rm[s:e], label = 'wspd running mean')
wspd_anom = test_wspd_r[s:e]-test_wspd_m_r[s:e]

axs[1].plot(yrs[s:e],wspd_anom, linewidth = 2, color = 'k',\
            label = 'wspd ANOMALY \n wspd - (monthly mean)')
norm_cflx_anom = cflx_anom/(np.nanmax(np.abs(cflx_anom)))
norm_wspd_anom = wspd_anom/(np.nanmax(np.abs(wspd_anom)))
axs[2].plot(yrs[s:e],norm_cflx_anom, linewidth = 2, color = 'r',\
            label = 'NORMALIZED TO 1 \n CFLX ANOMALY \n cflx - (monthly mean + running mean)')

axs[2].plot(yrs[s:e],norm_wspd_anom, linewidth = 2, color = 'k',\
            label = 'NORMALIZED TO 1 \n wspd ANOMALY \n wspd - (monthly mean + running mean)')

for i in range(0,3):
    axs[i].axhline(y=0,color ='k')
    axs[i].grid()
    axs[i].set_xlim([1970,2020])
    axs[i].legend()

In [ ]:
## create 

In [ ]:
Cflx_1AS1[Cflx_1AS1==0] = np.nan
Cflx_1AS1_bylat = np.nanmean(Cflx_1AS1,axis = 3)
Cflx_1AS1_monthly =(np.nanmean(np.nanmean(Cflx_1AS1 ,axis = 0), axis = 2))
Cflx_1AS1_bylat_anom = np.zeros_like(Cflx_1AS1_bylat)
for i in range(0,161):
    Cflx_1AS1_bylat_anom[i,:,:] = Cflx_1AS1_bylat[i,:,:] - Cflx_1AS1_monthly
##     
Cflx_WE43[Cflx_WE43==0] = np.nan
Cflx_WE43_bylat = np.nanmean(Cflx_WE43,axis = 3)
Cflx_WE43_monthly =(np.nanmean(np.nanmean(Cflx_WE43 ,axis = 0), axis = 2))
Cflx_WE43_bylat_anom = np.zeros_like(Cflx_WE43_bylat)
for i in range(0,161):
    Cflx_WE43_bylat_anom[i,:,:] = Cflx_WE43_bylat[i,:,:] - Cflx_WE43_monthly
    
Cflx_1AS1_bylat_r = np.zeros([161*12,149])
Cflx_1AS1_bylat_anom_r = np.zeros([161*12,149])
Cflx_WE43_bylat_r = np.zeros([161*12,149])
Cflx_WE43_bylat_anom_r = np.zeros([161*12,149])
ind = 0
for i in range(0,161):
    Cflx_1AS1_bylat_r[ind:ind+12,:] = Cflx_1AS1_bylat[i,:,:]
    Cflx_1AS1_bylat_anom_r[ind:ind+12,:] = Cflx_1AS1_bylat_anom[i,:,:]
    Cflx_WE43_bylat_r[ind:ind+12,:] = Cflx_WE43_bylat[i,:,:]
    Cflx_WE43_bylat_anom_r[ind:ind+12,:] = Cflx_WE43_bylat_anom[i,:,:]
    ind = ind+12

wspd_1AS1[wspd_1AS1==0] = np.nan
wspd_1AS1_bylat = np.nanmean(wspd_1AS1,axis = 3)
wspd_1AS1_monthly =(np.nanmean(np.nanmean(wspd_1AS1 ,axis = 0), axis = 2))
wspd_1AS1_bylat_anom = np.zeros_like(wspd_1AS1_bylat)
for i in range(0,161):
    wspd_1AS1_bylat_anom[i,:,:] = wspd_1AS1_bylat[i,:,:] - wspd_1AS1_monthly
##     
wspd_ERA5[wspd_ERA5==0] = np.nan
wspd_ERA5_bylat = np.nanmean(wspd_ERA5,axis = 3)
wspd_ERA5_monthly =(np.nanmean(np.nanmean(wspd_ERA5 ,axis = 0), axis = 2))
wspd_ERA5_bylat_anom = np.zeros_like(wspd_ERA5_bylat)
for i in range(0,161):
    wspd_ERA5_bylat_anom[i,:,:] = wspd_ERA5_bylat[i,:,:] - wspd_ERA5_monthly
    
wspd_1AS1_bylat_r = np.zeros([161*12,37])
wspd_1AS1_bylat_anom_r = np.zeros([161*12,37])
wspd_ERA5_bylat_r = np.zeros([161*12,37])
wspd_ERA5_bylat_anom_r = np.zeros([161*12,37])
ind = 0
for i in range(0,161):
    wspd_1AS1_bylat_r[ind:ind+12,:] = wspd_1AS1_bylat[i,:,:]
    wspd_1AS1_bylat_anom_r[ind:ind+12,:] = wspd_1AS1_bylat_anom[i,:,:]
    wspd_ERA5_bylat_r[ind:ind+12,:] = wspd_ERA5_bylat[i,:,:]
    wspd_ERA5_bylat_anom_r[ind:ind+12,:] = wspd_ERA5_bylat_anom[i,:,:]
    ind = ind+12

In [ ]:
lats = tmesh['nav_lat'][0:37,0]
yrs = np.arange(1940,2101,(1/12))
print(np.shape(yrs))
# w = plt.pcolormesh(yrs, lats, np.transpose(Cflx_stor), cmap = cm.cm.balance_r)
# plt.colorbar(w)
# plt.set_xlim([1950,2020])
fact = 0.6
tvmin = -1.5e-7; tvmax = 1.5e-7; tcmap = cm.cm.balance_r
fig, axs = plt.subplots(2,1, figsize=(24*fact, 18*fact), \
                        facecolor='w', edgecolor='k')
axs = axs.ravel()
w = axs[0].pcolormesh(yrs, lats, np.transpose(Cflx_1AS1_bylat_r[:,0:37]), \
                  cmap = tcmap, vmin = tvmin, vmax = tvmax)
# fig.colorbar(w, ax=axs[0], orientation = 'horizontal', label = 'mol/m2/s')
fig.colorbar(w, ax=axs[0], orientation = 'horizontal', label = 'mol/m2/s')

w = axs[1].pcolormesh(yrs, lats, np.transpose(Cflx_WE43_bylat_r[:,0:37]), \
                  cmap = tcmap, vmin = tvmin, vmax = tvmax)
fig.colorbar(w, ax=axs[1], orientation = 'horizontal', label = 'mol/m2/s')

tits = ['1AS1 - metoffice','WE43']
for i in range(0,2):
    axs[i].set_xlim([1970,2020])
    axs[0].grid()
    axs[i].set_title(tits[i])
    axs[i].set_ylabel('approx. lat.')
plt.suptitle('Hovmollers of mean air-sea co2 flux')
plt.tight_layout()

In [ ]:
lats = tmesh['nav_lat'][0:37,0]
yrs = np.arange(1940,2101,(1/12))
print(np.shape(yrs))
# w = plt.pcolormesh(yrs, lats, np.transpose(Cflx_stor), cmap = cm.cm.balance_r)
# plt.colorbar(w)
# plt.set_xlim([1950,2020])
fact = 0.6
tvmin = -0.5e-7; tvmax = 0.5e-7; tcmap = cm.cm.balance_r
fig, axs = plt.subplots(2,1, figsize=(24*fact, 18*fact), \
                        facecolor='w', edgecolor='k')
axs = axs.ravel()
w = axs[0].pcolormesh(yrs, lats, np.transpose(Cflx_1AS1_bylat_anom_r[:,0:37]), \
                  cmap = tcmap, vmin = tvmin, vmax = tvmax)
# fig.colorbar(w, ax=axs[0], orientation = 'horizontal', label = 'mol/m2/s')
fig.colorbar(w, ax=axs[0], orientation = 'horizontal', label = 'mol/m2/s')

w = axs[1].pcolormesh(yrs, lats, np.transpose(Cflx_WE43_bylat_anom_r[:,0:37]), \
                  cmap = tcmap, vmin = tvmin, vmax = tvmax)
fig.colorbar(w, ax=axs[1], orientation = 'horizontal', label = 'mol/m2/s')

tits = ['1AS1 - metoffice','WE43']
for i in range(0,2):
    axs[i].set_xlim([1970,2020])
    axs[0].grid()
    axs[i].set_title(tits[i])
    axs[i].set_ylabel('approx. lat.')
plt.suptitle('Hovmollers of mean air-sea co2 flux')
plt.tight_layout()

In [ ]:
lats = tmesh['nav_lat'][0:37,0]
yrs = np.arange(1940,2101,(1/12))
print(np.shape(yrs))
# w = plt.pcolormesh(yrs, lats, np.transpose(wspd_stor), cmap = cm.cm.balance_r)
# plt.colorbar(w)
# plt.set_xlim([1950,2020])
fact = 0.6
fig, axs = plt.subplots(2,1, figsize=(24*fact, 18*fact), \
                        facecolor='w', edgecolor='k')
axs = axs.ravel()
w = axs[0].pcolormesh(yrs, lats, np.transpose(wspd_1AS1_bylat_r[:,0:37]), \
                  cmap = cm.cm.deep, vmin = 2, vmax = 12)
# fig.colorbar(w, ax=axs[0], orientation = 'horizontal', label = 'mol/m2/s')
fig.colorbar(w, ax=axs[0], orientation = 'horizontal', label = 'mol/m2/s')

w = axs[1].pcolormesh(yrs, lats, np.transpose(wspd_ERA5_bylat_r[:,0:37]), \
                  cmap = cm.cm.deep, vmin = 2, vmax = 12)
fig.colorbar(w, ax=axs[1], orientation = 'horizontal', label = 'mol/m2/s')

tits = ['1AS1 - metoffice','ERA5']
for i in range(0,2):
    axs[i].set_xlim([1970,2020])
    axs[0].grid()
    axs[i].set_title(tits[i])
    axs[i].set_ylabel('approx. lat.')
plt.suptitle('Hovmollers of mean windspeed')
plt.tight_layout()

In [ ]:
lats = tmesh['nav_lat'][0:37,0]
yrs = np.arange(1940,2101,(1/12))
print(np.shape(yrs))
# w = plt.pcolormesh(yrs, lats, np.transpose(wspd_stor), cmap = cm.cm.balance_r)
# plt.colorbar(w)
# plt.set_xlim([1950,2020])
fact = 0.6
fig, axs = plt.subplots(2,1, figsize=(24*fact, 18*fact), \
                        facecolor='w', edgecolor='k')
axs = axs.ravel()
w = axs[0].pcolormesh(yrs, lats, np.transpose(wspd_1AS1_bylat_anom_r[:,0:37]), \
                  cmap = cm.cm.curl, vmin = -3, vmax = 3)
fig.colorbar(w, ax=axs[0], orientation = 'horizontal', label = 'm/s')

w = axs[1].pcolormesh(yrs, lats, np.transpose(wspd_ERA5_bylat_anom_r[:,0:37]), \
                  cmap = cm.cm.curl, vmin = -3, vmax = 3)
fig.colorbar(w, ax=axs[1], orientation = 'horizontal', label = 'm/s')

tits = ['1AS1 - metoffice','ERA5']
for i in range(0,2):
    axs[i].set_xlim([1970,2020])
    axs[0].grid()
    axs[i].set_title(tits[i])
    axs[i].set_ylabel('approx. lat.')
plt.suptitle('Hovmollers of mean windspeed anomaly' )
plt.tight_layout()

In [ ]:
wspd_1AS1_bylat_anom_r
wspd_ERA5_bylat_anom_r
wspd_1AS1_bylat_r
wspd_ERA5_bylat_r
Cflx_1AS1_bylat_anom_r
Cflx_WE43_bylat_anom_r
Cflx_1AS1_bylat_r
Cflx_WE43_bylat_r
Cflx_1AS1_monthly

In [ ]:
print(np.shape(Cflx_1AS1_monthly))
print(np.shape(Cflx_1AS1_bylat_r))
Cflx_1AS1_bylat_r_detrend = np.zeros([1932,37])
wspd_1AS1_bylat_r_detrend = np.zeros([1932,37])
Cflx_WE43_bylat_r_detrend = np.zeros([1932,37])
wspd_ERA5_bylat_r_detrend = np.zeros([1932,37])
for i in range(6,(1932-6)):
    parser = Cflx_1AS1_bylat_r[i-6:i+6,0:37]
    parser = np.nanmean(parser,axis = 0)
    Cflx_1AS1_bylat_r_detrend[i,:] = Cflx_1AS1_bylat_anom_r[i,0:37]-parser
    parser = Cflx_WE43_bylat_r[i-6:i+6,0:37]
    parser = np.nanmean(parser,axis = 0)
    Cflx_WE43_bylat_r_detrend[i,:] = Cflx_WE43_bylat_anom_r[i,0:37]-parser
    
    
    parser = wspd_1AS1_bylat_r[i-6:i+6,0:37]
    parser = np.nanmean(parser,axis = 0)
    wspd_1AS1_bylat_r_detrend[i,:] = wspd_1AS1_bylat_anom_r[i,0:37]-parser

    parser = wspd_ERA5_bylat_r[i-6:i+6,0:37]
    parser = np.nanmean(parser,axis = 0)
    wspd_ERA5_bylat_r_detrend[i,:] = wspd_ERA5_bylat_anom_r[i,0:37]-parser

In [ ]:
Cflx_1AS1_bylat_r_detrend[40,:]

In [ ]:
lats = tmesh['nav_lat'][0:37,0]
yrs = np.arange(1940,2101,(1/12))
print(np.shape(yrs))
# w = plt.pcolormesh(yrs, lats, np.transpose(wspd_stor), cmap = cm.cm.balance_r)
# plt.colorbar(w)
# plt.set_xlim([1950,2020])
fact = 0.6
fig, axs = plt.subplots(2,1, figsize=(24*fact, 18*fact), \
                        facecolor='w', edgecolor='k')
axs = axs.ravel()
w = axs[0].pcolormesh(yrs, lats, np.transpose(wspd_1AS1_bylat_r_detrend[:,0:37]), \
                  cmap = cm.cm.curl, vmin = -3, vmax = 3)
fig.colorbar(w, ax=axs[0], orientation = 'horizontal', label = 'm/s')

w = axs[1].pcolormesh(yrs, lats, np.transpose(wspd_ERA5_bylat_r_detrend[:,0:37]), \
                  cmap = cm.cm.curl, vmin = -3, vmax = 3)
fig.colorbar(w, ax=axs[1], orientation = 'horizontal', label = 'm/s')

tits = ['1AS1 - metoffice','ERA5']
for i in range(0,2):
    axs[i].set_xlim([1970,2020])
    axs[0].grid()
    axs[i].set_title(tits[i])
    axs[i].set_ylabel('approx. lat.')
plt.suptitle('Hovmollers of mean windspeed')
plt.tight_layout()

In [ ]:
wspd_ERA5_bylat_r_detrend